In [1]:
import pymysql
import requests
import pandas as pd
import numpy as np

### Genera un df con el archivo `csv` "DRGCODES"

In [3]:
df = pd.read_csv('C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\DRGCODES.csv', encoding='UTF-8', delimiter=',', decimal='.')
df.head(3)

,row_id,subject_id,hadm_id,drg_type,drg_code,description,drg_severity,drg_mortality
0,1338,10130,156668,HCFA,148,MAJOR SMALL & LARGE BOWEL PROCEDURES WITH COMP...,NaN,NaN
1,2188,10114,167957,HCFA,518,PERCUTANEOUS CARDIOVASCULAR PROCEDURES WITHOUT...,NaN,NaN
2,2599,10117,187023,HCFA,185,DENTAL & ORAL DIS EXCEPT EXTRACTIONS & RESTORA...,NaN,NaN


In [4]:
df = df.replace(np.nan, 0)

### Se conecta con la BD MySQL AWS

In [ ]:
con = pymysql.connect(
      host='database-proyecto-final.cpjrl88bhdul.us-east-1.rds.amazonaws.com',
      database='databaseuci',
      user='admin',
      password='datascience'
)

### Inserta los registros del df "DRGCODES" en la BD

In [5]:
sql_insert = ''' INSERT 
                   INTO  DRGCODES
                         (row_id, subject_id, hadm_id, drg_type, drg_code, description, drg_severity, drg_mortality)
                 VALUES  (%s, %s, %s, %s, %s, %s, %s, %s) '''
num_reng = 0
cursor = con.cursor()
for i in range(0, len(df)):
    row = int(df.loc[i, 'row_id'])
    sbid = int(df.loc[i, 'subject_id'])
    hadm = int(df.loc[i, 'hadm_id'])
    drtp = df.loc[i, 'drg_type']
    drcd = df.loc[i, 'drg_code']
    desc = df.loc[i, 'description']
    drsv = int(df.loc[i, 'drg_severity'])
    drmr = int(df.loc[i, 'drg_mortality'])
    reg_s = (row, sbid, hadm, drtp, drcd, desc, drsv, drmr)
    cursor.execute(sql_insert, reg_s)
    num_reng += 1
    if num_reng == 100:
        con.commit()
        num_reng = 0

### Confirma la inserción de los registros a la BD

In [6]:
con.commit()

### Cierra el cursor y la conexion a la BD

In [7]:
cursor.close()
con.close()